# Permutation tests
This notebook generates permutation-test results used in the thesis:
- Within-condition: above-chance decoding per subject × condition
- Between-condition: Classic vs Gabor differences per subject × contrast

Before running:
- Set the `ROOT` path below to your local directory

In [2]:
import os, sys, importlib
import numpy as np
import pandas as pd
import pickle
from decoding_utils import run_permutation_tests  # within-condition
import decoding_utils

ROOT = r"C:\Users\donja\Desktop\Thesis"

CODE_DIR      = os.path.join(ROOT, "Code")
DATA_DERIV    = os.path.join(ROOT, "Data", "derivatives")
RESULTS_DEC   = os.path.join(ROOT, "Results", "decoding")
RESULTS_PERM  = os.path.join(ROOT, "Results", "permutations")

os.makedirs(RESULTS_PERM, exist_ok=True)

print("CODE_DIR     :", CODE_DIR)
print("DATA_DERIV   :", DATA_DERIV)
print("RESULTS_DEC  :", RESULTS_DEC)
print("RESULTS_PERM :", RESULTS_PERM)

if CODE_DIR not in sys.path:
    sys.path.append(CODE_DIR)

importlib.reload(decoding_utils)

metrics_path = os.path.join(RESULTS_DEC, "decoding_metrics_all_subjects.csv")
df_metrics = pd.read_csv(metrics_path)

df_metrics = df_metrics[df_metrics["subject"] != "sub-004"]

print("df_metrics shape:", df_metrics.shape)
print("Subjects in metrics:", df_metrics["subject"].unique())
df_metrics.head()

CODE_DIR     : C:\Users\donja\Desktop\Thesis\Code
DATA_DERIV   : C:\Users\donja\Desktop\Thesis\Data\derivatives
RESULTS_DEC  : C:\Users\donja\Desktop\Thesis\Results\decoding
RESULTS_PERM : C:\Users\donja\Desktop\Thesis\Results\permutations
df_metrics shape: (48, 7)
Subjects in metrics: ['sub-002' 'sub-003' 'sub-005' 'sub-006']


,subject,file,stim_type,contrast,accuracy,decision_time,itr
0,sub-002,sub-002_cvep_classic_10.npz,classic,10,0.842857,4.2,51.343616
1,sub-002,sub-002_cvep_classic_100.npz,classic,100,0.966667,4.2,66.057397
2,sub-002,sub-002_cvep_classic_20.npz,classic,20,0.966667,4.2,66.057397
3,sub-002,sub-002_cvep_classic_30.npz,classic,30,1.000000,4.2,71.428571
4,sub-002,sub-002_cvep_classic_40.npz,classic,40,1.000000,4.2,71.428571


In [ ]:
# Change to subject you want to run
SUBJECTS_TO_RUN = ["sub-006"]

N_PERM_WITHIN = 1000

In [ ]:
within_summary_path = os.path.join(RESULTS_PERM, "within_perm_summary.csv")
within_nulls_path   = os.path.join(RESULTS_PERM, "within_perm_nulls.pkl")

all_new_summary_rows = []
all_new_null_rows = []

for subj in SUBJECTS_TO_RUN:
    print(f"\n Running within-condition permutations for {subj}")

    df_sub = df_metrics[df_metrics["subject"] == subj].copy()
    if df_sub.empty:
        print(f"  No rows found for {subj}, skipping.")
        continue

    df_with_p_sub, df_perm_sub = run_permutation_tests(
        df_sub,
        base_dir=DATA_DERIV,
        n_perm=N_PERM_WITHIN,
    )

    all_new_summary_rows.append(df_with_p_sub)
    all_new_null_rows.append(df_perm_sub)

if not all_new_summary_rows:
    print("\nNo new subjects processed, nothing to save.")
else:
    new_summary = pd.concat(all_new_summary_rows, ignore_index=True)
    new_nulls   = pd.concat(all_new_null_rows, ignore_index=True)

    if os.path.exists(within_summary_path):
        old_summary = pd.read_csv(within_summary_path)
        old_summary = old_summary[~old_summary["subject"].isin(SUBJECTS_TO_RUN)]
        summary_merged = pd.concat([old_summary, new_summary], ignore_index=True)
    else:
        summary_merged = new_summary

    summary_merged.to_csv(within_summary_path, index=False)
    print("\nUpdated summary file:", within_summary_path)
    print("Summary shape:", summary_merged.shape)

    if os.path.exists(within_nulls_path):
        old_nulls = pd.read_pickle(within_nulls_path)
        old_nulls = old_nulls[~old_nulls["subject"].isin(SUBJECTS_TO_RUN)]
        nulls_merged = pd.concat([old_nulls, new_nulls], ignore_index=True)
    else:
        nulls_merged = new_nulls

    nulls_merged.to_pickle(within_nulls_path)
    print("Updated nulls file:", within_nulls_path)
    print("Nulls shape:", nulls_merged.shape)

In [ ]:
from decoding_utils import permutation_test_classic_vs_grating

between_csv      = os.path.join(RESULTS_PERM, "between_perm_per_subject.csv")
between_nulls_pkl = os.path.join(RESULTS_PERM, "between_perm_per_subject_nulls.pkl")

all_new_between_rows = []
all_new_between_null_rows = []

contrasts = sorted(df_metrics["contrast"].unique())

for subj in SUBJECTS_TO_RUN:
    print(f"\n Running between-condition permutations (classic vs grating) for {subj} ")

    for c in contrasts:
        fn_classic = os.path.join(
            DATA_DERIV, subj, f"{subj}_cvep_classic_{c}.npz"
        )
        fn_grating = os.path.join(
            DATA_DERIV, subj, f"{subj}_cvep_grating_{c}.npz"
        )

        if not (os.path.exists(fn_classic) and os.path.exists(fn_grating)):
            print(f"  Missing data for {subj}, contrast {c}, skipping.")
            continue

        print(f"  {subj}, contrast {c}: running permutations...")
        d_real, p_value, null_diffs = permutation_test_classic_vs_grating(
            fn_classic,
            fn_grating,
            n_perm=N_PERM_WITHIN,
        )

        all_new_between_rows.append({
            "subject": subj,
            "contrast": c,
            "difference_grating_minus_classic": d_real,
            "p_value": p_value,
            "n_perm": N_PERM_WITHIN,
        })
        all_new_between_null_rows.append({
            "subject": subj,
            "contrast": c,
            "null_diffs": null_diffs,
        })

if all_new_between_rows:
    new_between = pd.DataFrame(all_new_between_rows)
    new_between_nulls = pd.DataFrame(all_new_between_null_rows)

    # Merge CSV
    if os.path.exists(between_csv):
        old_between = pd.read_csv(between_csv)
        old_between = old_between[~old_between["subject"].isin(SUBJECTS_TO_RUN)]
        between_merged = pd.concat([old_between, new_between], ignore_index=True)
    else:
        between_merged = new_between

    between_merged.to_csv(between_csv, index=False)
    print("\nUpdated between-condition CSV:", between_csv)
    print("Between CSV shape:", between_merged.shape)

    # Merge nulls pkl
    if os.path.exists(between_nulls_pkl):
        old_bn = pd.read_pickle(between_nulls_pkl)
        old_bn = old_bn[~old_bn["subject"].isin(SUBJECTS_TO_RUN)]
        between_nulls_merged = pd.concat([old_bn, new_between_nulls], ignore_index=True)
    else:
        between_nulls_merged = new_between_nulls

    between_nulls_merged.to_pickle(between_nulls_pkl)
    print("Updated between-condition nulls:", between_nulls_pkl)
    print("Between nulls shape:", between_nulls_merged.shape)
else:
    print("\nNo between-condition results computed in this run.")

In [30]:
# Make one-sided p-value two-sided

with open(os.path.join(RESULTS_PERM, "between_perm_per_subject_nulls.pkl"), "rb") as f:
    df_nulls = pickle.load(f)   # this is a DataFrame

# load observed differences
df_obs = pd.read_csv(
    os.path.join(RESULTS_PERM, "between_perm_per_subject.csv")
)

between = df_nulls.merge(
    df_obs[[
        "subject",
        "contrast",
        "difference_grating_minus_classic",
        "p_value"   # old one-sided p-value
    ]],
    on=["subject", "contrast"],
    how="left"
)

between[["subject", "contrast", "difference_grating_minus_classic"]].head()

between["p_two_sided"] = between.apply(
    lambda r: np.mean(
        np.abs(np.array(r["null_diffs"])) >=
        np.abs(r["difference_grating_minus_classic"])
    ),
    axis=1
)

between[[
    "subject",
    "contrast",
    "difference_grating_minus_classic",
    "p_value",        
    "p_two_sided"   
]]

between["sig"] = between["p_two_sided"] < 0.05
between[between["sig"]]

out = between.copy()
out.to_csv(
    os.path.join(RESULTS_PERM, "between_perm_per_subject_two_sided.csv"),
    index=False
)